In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
import cone_prog_refine as cpr 
import time

import cvxpy as cvx
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
n = 10

A = cvx.Variable((n,n))
B = np.random.randn(n,n)
obj = cvx.norm(A - B)

constr = [
    A == A.T,
    A >=0,
    A >> 0,
    cvx.log_det(A) >= 2.,
]

problem = cvx.Problem(cvx.Minimize(obj), constr)

In [3]:
problem.solve(verbose=True)
mosek_opt = problem.value

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1845, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 641, constraints m = 1416
Cones:	primal zero / dual free vars: 755
	linear vars: 101
	sd vars: 530, sd blks: 4
	exp vars: 30, dual exp vars: 0
Setup time: 3.01e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.79e+20  1.92e+20  1.00e+00 -8.34e+20  8.15e+20  4.18e+20  7.48e-03 
   100| 3.24e-03  9.37e-04  6.25e-05  4.52e+00  4.52e+00  5.68e-17  6.03e-02 


In [4]:
np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)), \
np.log(np.linalg.det(A.value)) - 2

(0.1524933849741329,
 1.1406737570321878e-05,
 5.632755124174842e-06,
 5.857657167318564e-06)

In [5]:
# def refine(problem, ):
    

In [12]:
from cvxpy.reductions.solvers.solving_chain import construct_solving_chain

def cvxpy_scs_to_csr(scs_solution, data):

    z_scs = cpr.xsy2z(scs_solution['x'], scs_solution['s'], scs_solution['y'])

    A, b, c, dims = data['A'], data['b'], data['c'], data['dims']
    
    dims_dict = {}
    dims_dict['l'] = int(dims.nonpos)
    dims_dict['z'] = int(dims.zero)
    dims_dict['ep'] = int(dims.exp)
    dims_dict['q'] = list([int(el) for el in dims.soc])
    dims_dict['s'] = list([int(el) for el in dims.psd])
    
    
    return A, b, c, z_scs, dims_dict

def solve_and_refine(cvxpy_problem, iters = 2, lsqr_iters = 30, verbose = True, warm_start=True):
    solving_chain = construct_solving_chain(cvxpy_problem, solver = 'SCS')
    data, inverse_data = solving_chain.apply(cvxpy_problem)

    if warm_start:
        scs_solution = problem._solver_cache[list(problem._solver_cache.keys())[0]]
    else:
        scs_solution = solving_chain.solve_via_data(cvxpy_problem, 
                                                data=data, 
                                                warm_start=warm_start, 
                                                verbose=verbose,
                                                solver_opts = {})#'eps':1E-100})

        assert problem._solver_cache[list(problem._solver_cache.keys())[0]] is scs_solution
        
    ## HERE GOES REFINE
    A, b, c, z_scs, dims = cvxpy_scs_to_csr(scs_solution, data)
    refined_z, _, _ = cpr.refine(A,b,c, dims, z_scs, iters = iters, lsqr_iters=lsqr_iters, verbose=verbose)
    new_residual, u, v = cpr.residual_and_uv(refined_z, A, b, c, cpr.make_prod_cone_cache(dims))
    scs_solution['x'], scs_solution['s'], scs_solution['y'], tau, kappa = cpr.uv2xsytaukappa(u, v, A.shape[1])
    problem.unpack_results(scs_solution, solving_chain, inverse_data)

    return problem.value

In [13]:
#mosek_opt, new_opt

In [40]:
new_opt = solve_and_refine(problem, iters = 2, lsqr_iters=None, warm_start=False)
print((mosek_opt - new_opt)/ mosek_opt)
print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)),\
     np.log(np.linalg.det(A.value)) - 2)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1845, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 641, constraints m = 1416
Cones:	primal zero / dual free vars: 755
	linear vars: 101
	sd vars: 530, sd blks: 4
	exp vars: 30, dual exp vars: 0
Setup time: 1.06e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.79e+20  1.92e+20  1.00e+00 -8.34e+20  8.15e+20  4.18e+20  2.31e-03 
   100| 3.24e-03  9.37e-04  6.25e-05  4.52e+00  4.52e+00  5.68e-17  6.71e-02 


In [64]:
# new_opt = solve_and_refine(problem, iters = 2, lsqr_iters = 500, warm_start=True)
# print((mosek_opt - new_opt)/ mosek_opt)
# print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)))

In [67]:
# new_opt = solve_and_refine(problem, warm_start=True)
# print((mosek_opt - new_opt)/ mosek_opt)
# print(np.linalg.eigh(A.value)[0][0], np.linalg.norm(A.value - A.T.value), np.linalg.norm(np.minimum(A.value, 0.)))

In [58]:
# from cvxpy.reductions.solvers.solving_chain import construct_solving_chain

# solving_chain = construct_solving_chain(problem, solver = 'SCS')
# data, inverse_data = solving_chain.apply(problem)

# scs_solution = solving_chain.solve_via_data(problem, 
#                                             data=data, 
#                                             warm_start=True, 
#                                             verbose=True,
#                                             solver_opts = {'eps':1E-8})

# ## HERE GOES REFINE

# A, b, c, z_scs, dims = cvxpy_scs_to_csr(scs_solution, data)
# refined_z = cpr.refine(A,b,c, dims, z_scs, verbose=True)
# new_residual, u, v = cpr.residual_and_uv(refined_z, A, b, c, cpr.make_prod_cone_cache(dims))
# scs_solution['x'], scs_solution['s'], scs_solution['y'], \
#     tau, kappa = cpr.uv2xsytaukappa(u, v, A.shape[1])
# print(tau, kappa)

# cvxpy_solution = solving_chain.invert(scs_solution, inverse_data)

# problem.unpack_results(scs_solution, solving_chain, inverse_data)
# problem.value

In [14]:
data['dims']

In [23]:
refined_z = refine(A,b,c, dims, z_scs, verbose=True)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               Conic Solution Refinement              

it.    ||N(z)||_2       z[-1]   LSQR  btrks     time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	8.89e-08	1e+00	  0	0	0.00
1	1.04e-08	1e+00	  30	0	0.05
2	6.82e-09	1e+00	  30	0	0.10
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Max num. refinement iters reached.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [33]:
refined_z = refine(A,b,c, dims, refined_z, verbose=True)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               Conic Solution Refinement              

it.    ||N(z)||_2       z[-1]   LSQR  btrks     time
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0	2.90e-09	1e+00	  0	0	0.00
1	2.88e-09	1e+00	  30	0	0.04
2	2.87e-09	1e+00	  30	0	0.09
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Max num. refinement iters reached.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [184]:
problem.unpack_results?

In [122]:
my_solution

In [110]:
#solution

In [112]:
my_solution

In [97]:
data = probl.get_problem_data(solver='SCS')
solving_chain = data[1]

solving_chain.solve(probl, warm_start=True, solver_opts={}, verbose=True)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 8625, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2731, constraints m = 6795
Cones:	primal zero / dual free vars: 3600
	linear vars: 900
	sd vars: 2295, sd blks: 2
Setup time: 2.35e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.07e+20  4.38e+20  1.00e+00 -2.81e+21  3.15e+21  9.37e+20  3.86e-03 
   100| 1.54e-02  2.35e-03  1.28e-04  8.46e+00  8.46e+00  3.01e-16  1.35e-01 
   200| 4.09e-03  6.27e-04  2

In [90]:
A, b, c, dims = data[0]['A'], data[0]['b'], data[0]['c'], data[0]['dims']
dims_dict = {}
dims_dict['l'] = int(dims.nonpos)
dims_dict['z'] = int(dims.zero)
dims_dict['q'] = list([int(el) for el in dims.soc])
dims_dict['s'] = list([int(el) for el in dims.psd])

TypeError: get_problem_data() got an unexpected keyword argument 'warm_start'

In [88]:
x_con, s_con, y_con, info = solve(A,b,c,dims_dict,
      max_lsqr_iters=1000, max_iters=2,
      #refine_solver_time_ratio=1.,
      verbose=True,
     solver_options={'verbose':True, 'eps':1e-6, 'max_iters':1000})


----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 8625, CG tol ~ 1/iter^(2.00)
eps = 1.00e-06, alpha = 1.50, max_iters = 1000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 2731, constraints m = 6795
Cones:	primal zero / dual free vars: 3600
	linear vars: 900
	sd vars: 2295, sd blks: 2
Setup time: 2.74e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 6.07e+20  4.38e+20  1.00e+00 -2.81e+21  3.15e+21  9.37e+20  4.60e-03 
   100| 1.54e-02  2.35e-03  1.28e-04  8.46e+00  8.46e+00  3.01e-16  1.88e-01 
   200| 4.09e-03  6.27e-04  2

In [89]:
data

({'A': <6795x2731 sparse matrix of type '<class 'numpy.float64'>'
  	with 8625 stored elements in Compressed Sparse Column format>,
  'b': array([-0., -0., -0., ...,  0.,  0.,  0.]),
  'c': array([1., 0., 0., ..., 0., 0., 0.]),
  'dims': <cvxpy.reductions.solvers.conic_solvers.conic_solver.ConeDims at 0xb2ab0feb8>,
  'offset': array([0.])},
  ({546: Variable((30, 30)), 1161: Variable((1830, 1)), 1162: Variable(())},
   {546: Variable((30, 30)),
    1161: Variable((60, 60), PSD=True),
    1162: Variable(())},
   {547: 547, 549: 549, 551: 551, 1163: 1163, 1165: 1165, 1167: 1167}),
  {'dims': <cvxpy.reductions.solvers.conic_solvers.conic_solver.ConeDims at 0xb2ab0feb8>,
   'eq_constr': [Zero(Expression(AFFINE, UNKNOWN, (30, 30))),
    Zero(Expression(AFFINE, UNKNOWN, (30, 30))),
    Zero(Expression(AFFINE, UNKNOWN, (30, 30))),
    Zero(Expression(AFFINE, UNKNOWN, (30, 30)))],
   'offset': 0.0,
   'other_constr': [NonPos(Expression(AFFINE, UNKNOWN, (30, 30))),
    PSD(Expression(AFFINE, UN

In [ ]:
data[1].apply

In [ ]:
data[1].accepts

In [ ]:
data[1].accepts

In [ ]:
data[1].solve

In [55]:
dir(data[1])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'accepts',
 'apply',
 'invert',
 'reductions',
 'solve',
 'solve_via_data',
 'solver']

In [31]:
#(B-A).value